In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor,plot_importance
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import gc
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv(r"C:\Users\DSE LAB 22\Downloads\archive\For_modeling.csv")[:80000]

In [ ]:
X, y = df.drop("Duration", axis = 1), df.Duration

In [ ]:
df.head(10)

,Unnamed: 0,Duration,Distance,PLong,PLatd,DLong,DLatd,Haversine,Pmonth,Pday,...,Dmin,DDweek,Temp,Precip,Wind,Humid,Solar,Snow,GroundTemp,Dust
0,0,3,50,37.544666,126.888359,37.544666,126.888359,0.000000,1,1,...,4,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
1,1,24,7670,37.506199,127.003944,37.551250,127.035103,5.713529,1,1,...,25,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
2,2,8,1390,37.544590,127.057083,37.537014,127.061096,0.913702,1,1,...,9,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
3,3,8,1820,37.571102,127.023560,37.561447,127.034920,1.468027,1,1,...,10,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
4,4,4,850,37.573242,127.015907,37.565849,127.016403,0.823227,1,1,...,6,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
5,5,8,1800,37.571102,127.023560,37.561447,127.034920,1.468027,1,1,...,10,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
6,6,25,4770,37.593288,127.036667,37.582500,126.998535,3.567644,1,1,...,27,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
7,7,4,630,37.552914,126.836571,37.557968,126.838287,0.581982,1,1,...,6,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
8,8,3,950,37.459953,127.024742,37.455799,127.016289,0.877524,1,1,...,5,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
9,9,25,3900,37.527122,127.028717,37.539654,127.052589,2.524430,1,1,...,28,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0


In [ ]:
df.columns

Index(['Unnamed: 0', 'Duration', 'Distance', 'PLong', 'PLatd', 'DLong',
       'DLatd', 'Haversine', 'Pmonth', 'Pday', 'Phour', 'Pmin', 'PDweek',
       'Dmonth', 'Dday', 'Dhour', 'Dmin', 'DDweek', 'Temp', 'Precip', 'Wind',
       'Humid', 'Solar', 'Snow', 'GroundTemp', 'Dust'],
      dtype='object')

In [ ]:
df.isnull().sum()

Unnamed: 0    0
Duration      0
Distance      0
PLong         0
PLatd         0
DLong         0
DLatd         0
Haversine     0
Pmonth        0
Pday          0
Phour         0
Pmin          0
PDweek        0
Dmonth        0
Dday          0
Dhour         0
Dmin          0
DDweek        0
Temp          0
Precip        0
Wind          0
Humid         0
Solar         0
Snow          0
GroundTemp    0
Dust          0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  80000 non-null  int64  
 1   Duration    80000 non-null  int64  
 2   Distance    80000 non-null  int64  
 3   PLong       80000 non-null  float64
 4   PLatd       80000 non-null  float64
 5   DLong       80000 non-null  float64
 6   DLatd       80000 non-null  float64
 7   Haversine   80000 non-null  float64
 8   Pmonth      80000 non-null  int64  
 9   Pday        80000 non-null  int64  
 10  Phour       80000 non-null  int64  
 11  Pmin        80000 non-null  int64  
 12  PDweek      80000 non-null  int64  
 13  Dmonth      80000 non-null  int64  
 14  Dday        80000 non-null  int64  
 15  Dhour       80000 non-null  int64  
 16  Dmin        80000 non-null  int64  
 17  DDweek      80000 non-null  int64  
 18  Temp        80000 non-null  float64
 19  Precip      80000 non-nul

In [ ]:
df.describe().T



,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,80000.0,40636.479813,23441.291207,0.000000,20331.750000,40647.500000,60907.250000,81260.000000
Duration,80000.0,18.857087,20.550209,2.000000,6.000000,11.000000,23.000000,119.000000
Distance,80000.0,2765.965125,3090.628173,10.000000,1060.000000,1770.000000,3180.000000,33260.000000
PLong,80000.0,37.548330,0.039827,37.443340,37.520695,37.552914,37.573277,37.689720
PLatd,80000.0,126.983903,0.074086,126.798599,126.921951,126.978500,127.046890,127.177612
DLong,80000.0,37.548007,0.039773,37.443340,37.520695,37.551849,37.573112,37.689720
DLatd,80000.0,126.983858,0.074154,126.798599,126.921082,126.979660,127.047462,127.177612
Haversine,80000.0,1.578019,1.595580,0.000000,0.677500,1.106391,1.929956,21.323001
Pmonth,80000.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Pday,80000.0,7.341550,4.206969,1.000000,4.000000,7.000000,11.000000,15.000000


In [ ]:
data=df
data

,Unnamed: 0,Duration,Distance,PLong,PLatd,DLong,DLatd,Haversine,Pmonth,Pday,...,Dmin,DDweek,Temp,Precip,Wind,Humid,Solar,Snow,GroundTemp,Dust
0,0,3,50,37.544666,126.888359,37.544666,126.888359,0.000000,1,1,...,4,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
1,1,24,7670,37.506199,127.003944,37.551250,127.035103,5.713529,1,1,...,25,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
2,2,8,1390,37.544590,127.057083,37.537014,127.061096,0.913702,1,1,...,9,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
3,3,8,1820,37.571102,127.023560,37.561447,127.034920,1.468027,1,1,...,10,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
4,4,4,850,37.573242,127.015907,37.565849,127.016403,0.823227,1,1,...,6,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,81256,22,4520,37.479084,126.879044,37.494995,126.858253,2.548591,1,15,...,48,0,5.2,0.0,1.3,44.0,0.0,0.0,0.0,80.0
79996,81257,35,3040,37.463322,127.036041,37.477509,127.045174,1.771504,1,15,...,2,0,5.2,0.0,1.3,44.0,0.0,0.0,0.0,80.0
79997,81258,36,6050,37.557461,126.861458,37.577221,126.812195,4.866204,1,15,...,4,0,5.2,0.0,1.3,44.0,0.0,0.0,0.0,80.0
79998,81259,3,730,37.575493,126.978500,37.570808,126.976433,0.551881,1,15,...,30,0,5.2,0.0,1.3,44.0,0.0,0.0,0.0,80.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2)
# create model instance
bst = XGBRegressor(n_estimators=50, max_depth=8, learning_rate=0.1,subsample=1)
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)
mean_squared_error(y_test,preds)

90.53780876737746

In [ ]:
preds

array([15.486602, 13.525189, 42.850906, ..., 16.41908 , 11.040175,
        4.639003], dtype=float32)

In [ ]:
clf =GradientBoostingRegressor(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.5934336540682644

In [ ]:
preds_clf = clf.predict(X_test)
mean_squared_error(y_test,preds_clf)

174.45203361999936

In [ ]:
clf1 =CatBoostRegressor(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0:	learn: 16.7769083	total: 170ms	remaining: 16.8s
1:	learn: 15.7856171	total: 179ms	remaining: 8.75s
2:	learn: 15.1300815	total: 183ms	remaining: 5.91s
3:	learn: 14.8545871	total: 186ms	remaining: 4.46s
4:	learn: 14.4901309	total: 189ms	remaining: 3.59s
5:	learn: 14.3125757	total: 192ms	remaining: 3.01s
6:	learn: 14.2412845	total: 195ms	remaining: 2.6s
7:	learn: 14.1876258	total: 199ms	remaining: 2.28s
8:	learn: 14.0282337	total: 202ms	remaining: 2.04s
9:	learn: 13.9672958	total: 205ms	remaining: 1.85s
10:	learn: 13.9277731	total: 208ms	remaining: 1.69s
11:	learn: 13.8885853	total: 211ms	remaining: 1.55s
12:	learn: 13.8473868	total: 215ms	remaining: 1.44s
13:	learn: 13.8109162	total: 218ms	remaining: 1.34s
14:	learn: 13.7714311	total: 221ms	remaining: 1.25s
15:	learn: 13.7309169	total: 224ms	remaining: 1.18s
16:	learn: 13.7041808	total: 227ms	remaining: 1.11s
17:	learn: 13.6748646	total: 230ms	remaining: 1.05s
18:	learn: 13.6513963	total: 233ms	remaining: 995ms
19:	learn: 13.6065201	t

0.5934336540682644

In [ ]:
preds_clf1 =clf1.predict(X_test)
mean_squared_error(y_test,preds_clf1)

295.9093963386178

In [ ]:
clf2 =AdaBoostRegressor(n_estimators=100 ,random_state=0).fit(X_train, y_train)
clf2.score(X_test, y_test)

0.35003935028815314

In [ ]:
preds_clf2=clf2.predict(X_test)
mean_squared_error(y_test,preds_clf2)

286.3654452396253